In [1]:
from lusidtools.jupyter_tools import toggle_code

""" Creating portfolios with different tax lot management methods

This notebook demonstrates how to create transaction portfolios under different tax lot accounting methodologies.

Attributes
----------
transactions
holdings
taxlots
accounting
"""

toggle_code("Toggle Docstring")

# Taxlot Management


Portfolios in LUSID can be created using any of the following tax lot accounting methods:
- First in first out (FIFO)
- Last in first out (LIFO)
- Average cost (default in LUSID)
- Lowest cost first
- Highest cost first

These will determine how tax lots are used to update the cost basis of a transaction portfolio when booking various transactions.
Helpful KB articles:
- [What are the supported tax-lot accounting methods in LUSID?](https://support.lusid.com/knowledgebase/article/KA-01886/en-us)
- [How do I handle different tax lot accounting conventions?](https://support.lusid.com/knowledgebase/article/KA-01887/en-us)


## 1. Initial Setup
This section will set up the parameters and methods used in section 2 to compare accounting methods.

In [2]:
# Import LUSID
import lusid
import lusid.models as models
import pandas as pd

# Import Libraries
import pytz
from lusidjam import RefreshingToken
import json
import os
from datetime import datetime

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

pd.set_option('display.float_format', lambda x: f'{x:,.0f}')

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print('LUSID Environment Initialised')
print('LUSID version : ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID version :  0.6.10895.0


In [3]:
# Create necessary API factories
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
property_definitions_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)

# 2. Loading our Data

In [4]:
# Specify scope and load data
txns = pd.read_csv("data/taxlot_accounting_transactions.csv")
scope = "taxlot_management_examplenb"
txns

,transaction_id,instrument_uid,type,units,transaction_price,transaction_date,settlement_date,total_consideration,client_internal
0,txn_000,LUID_00000X3Z,FundsIn,"33,000,000,000",1,2020-03-04T00:00:00.0000000+00:00,2020-03-05T00:00:00.0000000+00:00,"33,000,000,000",cash_USD
1,txn_001,LUID_00003D4V,Buy,"1,000",10,2020-03-05T00:00:00.0000000+00:00,2020-03-08T00:00:00.0000000+00:00,"10,000,000,000",inst_AAPL
2,txn_002,LUID_00003D4V,Buy,"1,000",12,2020-03-06T00:00:00.0000000+00:00,2020-03-09T00:00:00.0000000+00:00,"12,000,000,000",inst_AAPL
3,txn_003,LUID_00003D4V,Buy,"1,000",11,2020-03-07T00:00:00.0000000+00:00,2020-03-10T00:00:00.0000000+00:00,"11,000,000,000",inst_AAPL
4,txn_004,LUID_00003D4V,Sell,"1,500",12,2020-03-08T00:00:00.0000000+00:00,2020-03-11T00:00:00.0000000+00:00,"18,750,000,000",inst_AAPL


## 2.1 Create portfolio
When creating the transaction portfolio in LUSID we pass in a parameter of the accounting method type to determine how the tax lots are calculated. If not specified, average cost is the default method used.

In [5]:
def create_upload_portfolio(name, accounting_method):
    try:
        created_date = "2010-01-01T00:00:00.000000+00:00"

        # Create request body
        portfolio_request = models.CreateTransactionPortfolioRequest(
            display_name=f"Tax Lot Management Example - {accounting_method}",
            code=name,
            accounting_method=accounting_method,
            base_currency="USD",
            created=created_date,
        )

        # Upload new portfolio to LUSID
        response = transaction_portfolios_api.create_portfolio(
            scope=scope, create_transaction_portfolio_request=portfolio_request
        )

        created = response.version.effective_from
        print(
        f"portfolio '{response.id.code}', in scope {scope} created effective from: "
        f"{created.year}/"
        f"{created.month}/"
        f"{created.day}")
    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

In [6]:
portfolio_name_avg = "tax-lot-avgcost"
create_upload_portfolio(portfolio_name_avg, "AverageCost")

portfolio_name_fifo = "tax-lot-fifo"
create_upload_portfolio(portfolio_name_fifo, "FirstInFirstOut")

portfolio_name_lifo = "tax-lot-lifo"
create_upload_portfolio(portfolio_name_lifo, "LastInFirstOut")

portfolio_name_high = "tax-lot-highestcost"
create_upload_portfolio(portfolio_name_high, "HighestCostFirst")

portfolio_name_low = "tax-lot-lowestcost"
create_upload_portfolio(portfolio_name_low, "LowestCostFirst")

Could not create a portfolio with id 'tax-lot-avgcost' because it already exists in scope 'taxlot_management_examplenb'.
Could not create a portfolio with id 'tax-lot-fifo' because it already exists in scope 'taxlot_management_examplenb'.
Could not create a portfolio with id 'tax-lot-lifo' because it already exists in scope 'taxlot_management_examplenb'.
Could not create a portfolio with id 'tax-lot-highestcost' because it already exists in scope 'taxlot_management_examplenb'.
Could not create a portfolio with id 'tax-lot-lowestcost' because it already exists in scope 'taxlot_management_examplenb'.


## 2.2 Create Instruments

In [7]:
# Create instrument
instr_name=  "Apple Inc"
client_internal = "inst_AAPL"
ticker= "AAPL"

batch_upsert_request = {
    "Apple_Equity": models.InstrumentDefinition(
        name="Apple_Inc",
        identifiers={ "ClientInternal": models.InstrumentIdValue(value=client_internal),
                      "Ticker": models.InstrumentIdValue(value=ticker)},
    )
}

# Upsert new instruments to LUSID
instrument_response = instruments_api.upsert_instruments(
    request_body=batch_upsert_request
)

# Check response was successful
if len(instrument_response.failed) > 0:
    raise AssertionError("Instruments upsert failed. Inspect response for more detail")

## 2.3 Upload transactions

In [8]:
def upload_transactions_to_portfolio(portfolio_name, transactions):
    # Upsert transactions
    transactions_request = []
    txn_response = []

    for row, transactions in txns.iterrows():

        if transactions["client_internal"].startswith("cash"):
            instrument_identifier = {"Instrument/default/Currency": "USD"}

        else:
            instrument_identifier = {
                    "Instrument/default/ClientInternal": transactions["client_internal"]
                }
        # build request body
        transactions_request.append(
            models.TransactionRequest(
                transaction_id=transactions["transaction_id"],
                type=transactions["type"],
                instrument_identifiers=instrument_identifier,
                transaction_date=transactions["transaction_date"],
                settlement_date=transactions["settlement_date"],
                units=transactions["units"],
                transaction_price=models.TransactionPrice(price=transactions["transaction_price"], type="Price"),
                total_consideration=models.CurrencyAndAmount(
                    amount=transactions["total_consideration"], currency="USD"
                ),
            )
        )

        # Make Upsert Transactions call to LUSID
        txn_response.append(
            transaction_portfolios_api.upsert_transactions(
                scope=scope, code=portfolio_name, transaction_request=transactions_request
            )
        )

    print(f"{len(txn_response)} transactions upserted")

In [9]:
upload_transactions_to_portfolio(portfolio_name_avg, txns)
upload_transactions_to_portfolio(portfolio_name_fifo, txns)
upload_transactions_to_portfolio(portfolio_name_lifo, txns)
upload_transactions_to_portfolio(portfolio_name_high, txns)
upload_transactions_to_portfolio(portfolio_name_low, txns)

5 transactions upserted
5 transactions upserted
5 transactions upserted
5 transactions upserted
5 transactions upserted


In [10]:
# Show transactions in chronological order
txns.sort_values('transaction_date',ascending=True).drop('instrument_uid', axis=1)

,transaction_id,type,units,transaction_price,transaction_date,settlement_date,total_consideration,client_internal
0,txn_000,FundsIn,"33,000,000,000",1,2020-03-04T00:00:00.0000000+00:00,2020-03-05T00:00:00.0000000+00:00,"33,000,000,000",cash_USD
1,txn_001,Buy,"1,000",10,2020-03-05T00:00:00.0000000+00:00,2020-03-08T00:00:00.0000000+00:00,"10,000,000,000",inst_AAPL
2,txn_002,Buy,"1,000",12,2020-03-06T00:00:00.0000000+00:00,2020-03-09T00:00:00.0000000+00:00,"12,000,000,000",inst_AAPL
3,txn_003,Buy,"1,000",11,2020-03-07T00:00:00.0000000+00:00,2020-03-10T00:00:00.0000000+00:00,"11,000,000,000",inst_AAPL
4,txn_004,Sell,"1,500",12,2020-03-08T00:00:00.0000000+00:00,2020-03-11T00:00:00.0000000+00:00,"18,750,000,000",inst_AAPL


# 3. Cost Basis Comparison
Holdings before the sell transaction displayed by tax lot show the 3 distinct tax lot transactions. We can see, by running the cells below, that in this example each tax lot has a different cost basis for the same number of units, therefore a different price per unit in each tax lot.

In [11]:
# Prints quick summary from a get_holdings() response of positions for a given effective_at date broken down by tax lots
def display_holding_positions_by_taxlot(effective_at, portfolio_name):
    # get holdings
    response = transaction_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_name,
    effective_at = effective_at.isoformat(),
    property_keys=["Instrument/default/Name"],
     by_taxlots	=True
    )

    #print(response)
    # inspect holdings response for today
    hld = [i for i in response.values]

    names = []
    cost = []
    units = []
    txnid = []

    for item in hld:
        #print(item)
        if item.holding_type_name == "Position":
            names.append(item.properties["Instrument/default/Name"].value.label_value)
            cost.append(item.cost.amount)
            units.append(item.units)
            txnid.append(item.properties["Holding/default/TaxlotId"].value.label_value)

    data = {"cost_basis": cost, "units": units, "transaction_id": txnid}

    summary = pd.DataFrame(data=data, index=names)
    return summary

In [12]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=7, tzinfo=pytz.UTC), portfolio_name_fifo)

,cost_basis,units,transaction_id
Apple_Inc,"10,000,000,000","1,000",txn_001
Apple_Inc,"12,000,000,000","1,000",txn_002
Apple_Inc,"11,000,000,000","1,000",txn_003


The following sections will look at the final holdings positions for portfolios using 5 different accounting methods after 1,500 units have been sold in a single sell transaction.

## 3.1 Average Cost
Average cost uses the average price of the shares, which in this example is $ \frac{33,000,000,000}{3,000} = 11,000.$
As shown by running the cell below, the portfolios are part of the same tax lot when using the *Average Cost* accounting method. The total cost basis after the sale will be calculated as $11,000 \times 1,500 = 16,500,000,000$.
This is the default accounting method for a portfolio created in LUSID.

In [13]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_name_avg)

,cost_basis,units,transaction_id
Apple_Inc,"16,500,000,000","1,500",txn_001


## 3.2 First In First Out (FIFO)
Using FIFO, the equities bought in the first buy transaction `txn_001` and 500 units of the equities bought in `txn_002` will be sold.
The display from the cell below, shows the remaining holdings with 500 units of the equities bought in `txn_002` and all the equities bought in `txn_003` in separate tax lots.

In [14]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_name_fifo)

,cost_basis,units,transaction_id
Apple_Inc,"6,000,000,000",500,txn_002
Apple_Inc,"11,000,000,000","1,000",txn_003


## 3.3 Last In Last Out (LIFO)
Using LIFO, the equities bought in the buy transaction `txn_003` and equities bought in `txn_002`. The display below shows the remaining cost basis which consists of 1,000 units of the equities bought in `txn_001` and 500 of the equities bought in `txn_002` in separate tax lots.
The difference in the cost basis compared to the **FIFO** method is due to the difference in total consideration between `txn_001` and `txn_003`.


In [15]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_name_lifo)

,cost_basis,units,transaction_id
Apple_Inc,"10,000,000,000","1,000",txn_001
Apple_Inc,"6,000,000,000",500,txn_002


## 3.4 Highest First
Using Highest First, the equities with the highest transaction price will be sold first. So in the sell transaction of 1,500 units, 1,000 units of equities from `txn_002` and 500 units of `txn_003`. The equities from `txn_001` and 500 units from `txn_003` remain in the cost basis as shown in the display below.

In [16]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_name_high)

,cost_basis,units,transaction_id
Apple_Inc,"10,000,000,000","1,000",txn_001
Apple_Inc,"5,500,000,000",500,txn_003


## 3.5 Lowest First
Using Lowest First, the equities with the lowest transaction price will be sold first. So in the sell transaction of 1,500 units,  1,000 of the equities in `txn_001` and 500 units of `txn_003`. The equities from `txn_002` and 500 units from `txn_003` remain in the cost basis as shown in the display below.
The difference in total consideration between `txn_001` and `txn_002` is show in the difference in cost basis compared to **Highest First** method.

In [17]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_name_low)

,cost_basis,units,transaction_id
Apple_Inc,"12,000,000,000","1,000",txn_002
Apple_Inc,"5,500,000,000",500,txn_003
